In [26]:
import pandas as pd
import numpy as np

In [27]:
users = pd.read_csv(
    r'E:\COURSES\Кодинг\[Yandex praxis]. DS specialist + (2022)\3\01 Исследовательский анализ данных\консультации\olist_customers_dataset.csv')
orders = pd.read_csv(
    r'E:\COURSES\Кодинг\[Yandex praxis]. DS specialist + (2022)\3\01 Исследовательский анализ данных\консультации\olist_orders_dataset.csv')
items = pd.read_csv(
    r'E:\COURSES\Кодинг\[Yandex praxis]. DS specialist + (2022)\3\01 Исследовательский анализ данных\консультации\olist_order_items_dataset.csv')

In [28]:
users.head(2)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP


In [29]:
orders.head(2)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00


In [30]:
items.head(2)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93


In [31]:
list(orders.order_status.unique())

['delivered',
 'invoiced',
 'shipped',
 'processing',
 'unavailable',
 'canceled',
 'created',
 'approved']

In [32]:
def cuter(value):
    return value[:7]


orders['month'] = orders.order_purchase_timestamp.apply(cuter)

## 2. Сколько заказов в месяц не доставляется по разным причинам

Первый вариант

In [33]:
task_1_var_1 = (
    orders
    [orders.order_status.isin(['canceled', 'unavailable'])]
    .groupby(['month', 'order_status'], as_index=False)
    .order_id
    .count()
    .groupby('order_status')
    .order_id.mean()
)

Первый вариант другое оформление

In [34]:
task_1_var_2 = (
    orders
    [orders.order_status.isin(['canceled', 'unavailable'])]
    .groupby(['month', 'order_status'], as_index=False)
    .order_id
    .count()
    .groupby('order_status')
    .order_id.mean()
)
print(task_1_var_2)

order_status
canceled       26.041667
unavailable    29.000000
Name: order_id, dtype: float64


Второй вариант

In [35]:
task_1_var_2_1 = (
    orders
    [orders.order_status.isin(['canceled', 'unavailable'])]
    .month.nunique()
)
# print(task_1_var_2_1)

task_1_var_2_2 = (
    orders
    [orders.order_status.isin(['canceled', 'unavailable'])]
    .groupby(['order_status'], as_index=False)
    .order_id
    .count()
)
# print(task_1_var_2_2)

task_1_var_2_2['final_answer'] = round(task_1_var_2_2['order_id'] / task_1_var_2_1, 2)
print(task_1_var_2_2)


  order_status  order_id  final_answer
0     canceled       625         26.04
1  unavailable       609         25.38


## 3. По каждому товару определить в какой день недели товар чаще всего покупается

In [36]:
def get_weekday(value):
    if not pd.isna(value):
        return pd.to_datetime(value).isocalendar().weekday

In [37]:
bad_statuses = ['created', 'canceled', 'unavailable']

In [38]:
task_3 = (orders.query('order_status not in @bad_statuses')[['order_id', 'order_approved_at']])
task_3['weekday'] = task_3.order_approved_at.apply(get_weekday).dropna()  # apply Обращется к Df построчно

print(task_3)

                               order_id    order_approved_at  weekday
0      e481f51cbdc54678b7cc49136f2d6af7  2017-10-02 11:07:15      1.0
1      53cdb2fc8bc7dce0b6741e2150273451  2018-07-26 03:24:27      4.0
2      47770eb9100c2d0c44946d9cf07ec65d  2018-08-08 08:55:23      3.0
3      949d5b44dbf5de918fe9c16f97b45f8a  2017-11-18 19:45:59      6.0
4      ad21c59c0840e6cb83a9ceb5573f8159  2018-02-13 22:20:29      2.0
...                                 ...                  ...      ...
99436  9c5dedf39a927c1b2549525ed64a053c  2017-03-09 09:54:05      4.0
99437  63943bddc261676b46f01ca7ac2f7bd8  2018-02-06 13:10:37      2.0
99438  83c1379a015df1e13d02aae0204711ab  2017-08-27 15:04:16      7.0
99439  11c177c8e97725db2631073c19f07b62  2018-01-08 21:36:21      1.0
99440  66dea50a8b16d9b4dee7af250b4be1a5  2018-03-09 11:20:28      5.0

[98202 rows x 3 columns]


In [39]:
task_3.isna().sum()

order_id              0
order_approved_at    14
weekday              14
dtype: int64

In [40]:
def most_frequent(series):
    if len(series) == 0:
        return None
    return series.value_counts().index[0]

In [41]:
items.merge(task_3, on='order_id', how='inner').groupby('product_id', as_index=False).transform(most_frequent)

IndexError: index 0 is out of bounds for axis 0 with size 0

## 4. Построй RFM-кластеры для пользователей. Выведи для каждого кластера средние значения метрик R,F,M. (Опиши, как были построены метрики R, F, M)

* R (Recency) — давность последней покупки: сколько времени прошло с момента последней транзакции клиента.
* F (Frequency) — частота покупок: сколько раз клиент совершал покупки за определённый период.
* M (Monetary) — денежный объём: общая сумма денег, которую клиент потратил за определённый период.

In [42]:
orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,month
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,2017-10
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,2018-07
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,2018-08
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,2017-11
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,2018-02
...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00,2017-03
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,2018-02
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,2017-08
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,2018-01


Frequency - суммарное количество покупок у пользователя за весь период данных.
1. Correspond customer_unique_id to customer_id
2. Group orders of each users

In [43]:
task_4_1 = orders.query('order_status == "delivered"')
print(task_4_1)

                               order_id                       customer_id  \
0      e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1      53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2      47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3      949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4      ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   
...                                 ...                               ...   
99436  9c5dedf39a927c1b2549525ed64a053c  39bd1228ee8140590ac3aca26f2dfe00   
99437  63943bddc261676b46f01ca7ac2f7bd8  1fca14ff2861355f6e5f14306ff977a7   
99438  83c1379a015df1e13d02aae0204711ab  1aa71eb042121263aafbe80c1b562c9c   
99439  11c177c8e97725db2631073c19f07b62  b331b74b18dc79bcdf6532d51e1637c1   
99440  66dea50a8b16d9b4dee7af250b4be1a5  edb027a75a1449115f6b43211ae02a24   

      order_status order_purchase_timestamp    order_approved_at  \
0      

In [79]:
# users[['customer_id','customer_unique_id']]
task_4_2 = ((users
                [['customer_id', 'customer_unique_id']]
                .merge(
                    orders.query('order_status == "delivered"'),
                    on='customer_id',
                    how='left'
                       )
                .groupby('customer_unique_id',
                        as_index=False)
            )
            .order_id
            .count()

            )
print(task_4_2)

                     customer_unique_id  order_id
0      0000366f3b9a7992bf8c76cfdf3221e2         1
1      0000b849f77a49e4a4ce2b2a4ca5be3f         1
2      0000f46a3911fa3c0805444483337064         1
3      0000f6ccb0745a6a4b88665a16c9f078         1
4      0004aac84e0df4da2b147fca70cf8255         1
...                                 ...       ...
96091  fffcf5a5ff07b0908bd4e2dbc735a684         1
96092  fffea47cd6d3cc0a88bd621562a9d061         1
96093  ffff371b4d645b6ecea244b27531430a         1
96094  ffff5962728ec6157033ef9805bacc48         1
96095  ffffd2657e2aad2907e67c3e9daecbeb         1

[96096 rows x 2 columns]


In [80]:
task_4_2.order_id.value_counts()

order_id
1     90557
0      2738
2      2573
3       181
4        28
5         9
6         5
7         3
9         1
15        1
Name: count, dtype: int64

In [81]:
np.digitize([0,1,4],[0,1,2])

array([1, 2, 3], dtype=int64)

In [82]:
task_4_2['frequency_cluster'] = np.digitize(task_4_2.order_id, [0, 1, 2])

In [83]:
task_4_2 = task_4_2.rename(columns={'order_id': 'orders_count'})

In [84]:
task_4_2

,customer_unique_id,orders_count,frequency_cluster
0,0000366f3b9a7992bf8c76cfdf3221e2,1,2
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,2
2,0000f46a3911fa3c0805444483337064,1,2
3,0000f6ccb0745a6a4b88665a16c9f078,1,2
4,0004aac84e0df4da2b147fca70cf8255,1,2
...,...,...,...
96091,fffcf5a5ff07b0908bd4e2dbc735a684,1,2
96092,fffea47cd6d3cc0a88bd621562a9d061,1,2
96093,ffff371b4d645b6ecea244b27531430a,1,2
96094,ffff5962728ec6157033ef9805bacc48,1,2


* cluster `0` - 0 orders
* cluster `1` - 1 order
* cluster `2` - > 1 orders

# Сделать самостоятельно классы для recency and monetory